In [8]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import numpy as np

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hanne\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
import pandas as pd

In [10]:
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [11]:
# You have to adjust the 
df_arguments = pd.read_csv("kpm_data/arguments_dev.csv")
df_keypoints = pd.read_csv("kpm_data/key_points_dev.csv")
df_labels = pd.read_csv("kpm_data/labels_dev.csv")

In [12]:
df_arguments.shape
df_arguments.head()

,arg_id,argument,topic,stance
0,arg_4_0,having a school uniform can reduce bullying as...,We should abandon the use of school uniform,-1
1,arg_4_1,it is a good way to instill discipline,We should abandon the use of school uniform,-1
2,arg_4_2,it is cheaper for parents to buy school unifor...,We should abandon the use of school uniform,-1
3,arg_4_3,one of the benefits of school uniforms is that...,We should abandon the use of school uniform,-1
4,arg_4_4,researchers found that in schools across the w...,We should abandon the use of school uniform,-1


In [13]:
key_points = df_keypoints['key_point']

In [14]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

In [15]:
key_point_embeddings = sbert_model.encode(key_points)
key_point_embeddings


array([[ 0.24013345, -0.60543454,  1.1649946 , ...,  0.7345633 ,
        -0.7821225 ,  0.4234572 ],
       [-0.4927498 , -0.4679714 ,  0.30425778, ...,  1.0996794 ,
        -0.6606649 ,  0.21055056],
       [-0.06403853, -0.14591816,  0.95404166, ...,  0.84960556,
        -0.9897014 ,  0.30558982],
       ...,
       [ 0.8822509 , -0.22543676,  0.19426186, ...,  0.6371331 ,
        -1.0707703 ,  0.22760639],
       [-0.20408638,  0.53391   ,  0.01523506, ..., -0.9874041 ,
        -1.0077103 , -0.09953481],
       [-0.5155356 ,  0.5542618 ,  0.58143574, ..., -0.32241064,
        -0.45623398, -0.1330981 ]], dtype=float32)

In [16]:
query_arg = df_arguments["argument"][1]
query_arg_id = df_arguments["arg_id"][1]

In [17]:
print(query_arg_id)

arg_4_1


In [18]:
query_vec = sbert_model.encode([query_arg])[0]

In [19]:
l_arg = []
l_sim = []
for kp in key_points:
  sim = cosine(query_vec, sbert_model.encode([kp])[0])
  l_arg.append(kp)
  l_sim.append(sim)
  # print("Sentence = ", sent, "; similarity = ", sim)
    
result = pd.DataFrame({"key_point": l_arg, "similarity": l_sim})

In [20]:
result.head()

,key_point,similarity
0,Children can still express themselves using ot...,0.457633
1,School uniform reduces bullying,0.551126
2,School uniforms encourage discipline or focus ...,0.674641
3,School uniforms saves costs,0.566534
4,School uniforms create a sense of equality/unity,0.630723


In [21]:
most_similar_kp = result.loc[result['similarity'].idxmax()]
most_similar_kp_id = df_keypoints[df_keypoints["key_point"]==most_similar_kp["key_point"]]["key_point_id"].iloc[0]
print(most_similar_kp_id)

kp_8_7


In [22]:
correct_kp = df_labels[df_labels["arg_id"]==query_arg_id]
correct_kp = correct_kp[correct_kp["label"]==1]["key_point_id"].iloc[0]
print(correct_kp)

kp_4_2


In [23]:
print("Query: ", query_arg)
print("Key_point is: ", most_similar_kp["key_point"])
if correct_kp == most_similar_kp_id:
    print("Matching: Correct")
else:
    print("Matching: False")

Query:  it is a good way to instill discipline
Key_point is:  Austerity is important to increase stability
Matching: False


In [24]:
argument_embeddings = sbert_model.encode(df_arguments['argument'])
argument_embeddings

array([[-0.21171936,  0.33552772,  0.3132487 , ...,  0.7327052 ,
        -0.82815576,  0.05489646],
       [-0.1577365 , -0.3815103 ,  2.0090349 , ...,  0.07120312,
        -1.2360226 ,  0.04574307],
       [-0.5563665 ,  0.18553925,  0.03002767, ...,  0.61501706,
        -0.04241766,  0.16210704],
       ...,
       [-0.19501917,  0.4722792 ,  1.3385419 , ..., -0.5832078 ,
        -0.84415907,  0.84149545],
       [-0.11743899,  0.7670163 ,  0.0462397 , ..., -0.6081743 ,
        -0.9796996 ,  0.5086879 ],
       [-0.02283251, -0.18045345,  1.710494  , ..., -0.5065564 ,
        -1.2832665 ,  0.58527386]], dtype=float32)

In [25]:
df_arguments


,arg_id,argument,topic,stance
0,arg_4_0,having a school uniform can reduce bullying as...,We should abandon the use of school uniform,-1
1,arg_4_1,it is a good way to instill discipline,We should abandon the use of school uniform,-1
2,arg_4_2,it is cheaper for parents to buy school unifor...,We should abandon the use of school uniform,-1
3,arg_4_3,one of the benefits of school uniforms is that...,We should abandon the use of school uniform,-1
4,arg_4_4,researchers found that in schools across the w...,We should abandon the use of school uniform,-1
...,...,...,...,...
927,arg_15_222,we should end it and give people that are the ...,We should end affirmative action,1
928,arg_15_223,we should end it so that every persons are equal,We should end affirmative action,1
929,arg_15_224,we should evaluate candidates based solely on ...,We should end affirmative action,1
930,arg_15_225,we should select people based on their merit o...,We should end affirmative action,1


In [26]:
df_keypoints



,key_point_id,key_point,topic,stance
0,kp_4_0,Children can still express themselves using ot...,We should abandon the use of school uniform,-1
1,kp_4_1,School uniform reduces bullying,We should abandon the use of school uniform,-1
2,kp_4_2,School uniforms encourage discipline or focus ...,We should abandon the use of school uniform,-1
3,kp_4_3,School uniforms saves costs,We should abandon the use of school uniform,-1
4,kp_4_4,School uniforms create a sense of equality/unity,We should abandon the use of school uniform,-1
5,kp_4_5,School uniform is harming the student's self e...,We should abandon the use of school uniform,1
6,kp_4_6,School uniforms are expensive,We should abandon the use of school uniform,1
7,kp_4_7,School uniforms are often uncomfortable/sexist,We should abandon the use of school uniform,1
8,kp_4_8,School uniform harms learning/creativity,We should abandon the use of school uniform,1
9,kp_4_9,School uniforms increase conformity or harm in...,We should abandon the use of school uniform,1


In [27]:
df_labels

,arg_id,key_point_id,label
0,arg_4_121,kp_4_5,1
1,arg_4_121,kp_4_6,0
2,arg_4_121,kp_4_7,0
3,arg_4_122,kp_4_8,0
4,arg_4_122,kp_4_5,1
...,...,...,...
3453,arg_15_226,kp_15_3,0
3454,arg_15_226,kp_15_4,0
3455,arg_15_226,kp_15_5,1
3456,arg_15_226,kp_15_6,0


In [28]:
result = pd.concat([df_labels, df_arguments], axis=1, join="inner")
result.head()

,arg_id,key_point_id,label,arg_id,argument,topic,stance
0,arg_4_121,kp_4_5,1,arg_4_0,having a school uniform can reduce bullying as...,We should abandon the use of school uniform,-1
1,arg_4_121,kp_4_6,0,arg_4_1,it is a good way to instill discipline,We should abandon the use of school uniform,-1
2,arg_4_121,kp_4_7,0,arg_4_2,it is cheaper for parents to buy school unifor...,We should abandon the use of school uniform,-1
3,arg_4_122,kp_4_8,0,arg_4_3,one of the benefits of school uniforms is that...,We should abandon the use of school uniform,-1
4,arg_4_122,kp_4_5,1,arg_4_4,researchers found that in schools across the w...,We should abandon the use of school uniform,-1


In [29]:
merged_dataset=df_labels.merge(df_arguments, left_on="arg_id", right_on="arg_id")

In [30]:
df_arguments.loc[df_arguments['arg_id'] == "arg_4_121"]

,arg_id,argument,topic,stance
121,arg_4_121,A real education is about giving students the ...,We should abandon the use of school uniform,1


In [31]:
df_labels

,arg_id,key_point_id,label
0,arg_4_121,kp_4_5,1
1,arg_4_121,kp_4_6,0
2,arg_4_121,kp_4_7,0
3,arg_4_122,kp_4_8,0
4,arg_4_122,kp_4_5,1
...,...,...,...
3453,arg_15_226,kp_15_3,0
3454,arg_15_226,kp_15_4,0
3455,arg_15_226,kp_15_5,1
3456,arg_15_226,kp_15_6,0


In [32]:
full_dataset=merged_dataset.merge(df_keypoints, left_on="key_point_id", right_on="key_point_id")
full_dataset

,arg_id,key_point_id,label,argument,topic_x,stance_x,key_point,topic_y,stance_y
0,arg_4_121,kp_4_5,1,A real education is about giving students the ...,We should abandon the use of school uniform,1,School uniform is harming the student's self e...,We should abandon the use of school uniform,1
1,arg_4_122,kp_4_5,1,children express themselves through the clothe...,We should abandon the use of school uniform,1,School uniform is harming the student's self e...,We should abandon the use of school uniform,1
2,arg_4_123,kp_4_5,1,"children should be able to dress as they wish,...",We should abandon the use of school uniform,1,School uniform is harming the student's self e...,We should abandon the use of school uniform,1
3,arg_4_124,kp_4_5,1,children should be allowed to express themselves,We should abandon the use of school uniform,1,School uniform is harming the student's self e...,We should abandon the use of school uniform,1
4,arg_4_126,kp_4_5,1,freedom of expression extends to the right to ...,We should abandon the use of school uniform,1,School uniform is harming the student's self e...,We should abandon the use of school uniform,1
...,...,...,...,...,...,...,...,...,...
3453,arg_15_218,kp_15_7,1,we should end affirmative action because stude...,We should end affirmative action,1,Affirmative action reduces quality,We should end affirmative action,1
3454,arg_15_219,kp_15_7,0,we should end affirmative action because this ...,We should end affirmative action,1,Affirmative action reduces quality,We should end affirmative action,1
3455,arg_15_223,kp_15_7,0,we should end it so that every persons are equal,We should end affirmative action,1,Affirmative action reduces quality,We should end affirmative action,1
3456,arg_15_225,kp_15_7,0,we should select people based on their merit o...,We should end affirmative action,1,Affirmative action reduces quality,We should end affirmative action,1


In [33]:
arguments = full_dataset["argument"]
arguments

0       A real education is about giving students the ...
1       children express themselves through the clothe...
2       children should be able to dress as they wish,...
3        children should be allowed to express themselves
4       freedom of expression extends to the right to ...
                              ...                        
3453    we should end affirmative action because stude...
3454    we should end affirmative action because this ...
3455     we should end it so that every persons are equal
3456    we should select people based on their merit o...
3457    yes, you should have to work your way to the t...
Name: argument, Length: 3458, dtype: object

In [34]:
small = full_dataset[:10]
small

,arg_id,key_point_id,label,argument,topic_x,stance_x,key_point,topic_y,stance_y
0,arg_4_121,kp_4_5,1,A real education is about giving students the ...,We should abandon the use of school uniform,1,School uniform is harming the student's self e...,We should abandon the use of school uniform,1
1,arg_4_122,kp_4_5,1,children express themselves through the clothe...,We should abandon the use of school uniform,1,School uniform is harming the student's self e...,We should abandon the use of school uniform,1
2,arg_4_123,kp_4_5,1,"children should be able to dress as they wish,...",We should abandon the use of school uniform,1,School uniform is harming the student's self e...,We should abandon the use of school uniform,1
3,arg_4_124,kp_4_5,1,children should be allowed to express themselves,We should abandon the use of school uniform,1,School uniform is harming the student's self e...,We should abandon the use of school uniform,1
4,arg_4_126,kp_4_5,1,freedom of expression extends to the right to ...,We should abandon the use of school uniform,1,School uniform is harming the student's self e...,We should abandon the use of school uniform,1
5,arg_4_129,kp_4_5,1,it is not fair to not allow children to expres...,We should abandon the use of school uniform,1,School uniform is harming the student's self e...,We should abandon the use of school uniform,1
6,arg_4_130,kp_4_5,0,Kids having to wear school uniforms is not fai...,We should abandon the use of school uniform,1,School uniform is harming the student's self e...,We should abandon the use of school uniform,1
7,arg_4_131,kp_4_5,1,kids should be able to express them selves wit...,We should abandon the use of school uniform,1,School uniform is harming the student's self e...,We should abandon the use of school uniform,1
8,arg_4_132,kp_4_5,1,kids should be allowed to wear what is comfort...,We should abandon the use of school uniform,1,School uniform is harming the student's self e...,We should abandon the use of school uniform,1
9,arg_4_133,kp_4_5,0,making kids wear uniforms that are all the sam...,We should abandon the use of school uniform,1,School uniform is harming the student's self e...,We should abandon the use of school uniform,1


In [35]:
test1=[]
for index,el in full_dataset.iterrows():
    row = [sbert_model.encode(el["argument"]), sbert_model.encode(el["key_point"])]
    np_row = np.array(row)
    row = np_row.flatten()
    test1.append(row)
    #print(el["argument"], el["key_point"], el["label"p

np_test_training = np.array(test1)
np_test_training

array([[-0.22546919,  0.42217124,  1.0890179 , ..., -0.05868514,
        -0.71161807, -0.01766074],
       [ 0.05293709,  0.05693178,  0.6169441 , ..., -0.05868514,
        -0.71161807, -0.01766074],
       [ 0.3017908 , -0.2542112 ,  0.69037217, ..., -0.05868514,
        -0.71161807, -0.01766074],
       ...,
       [-0.00661286,  0.23289303,  1.4282135 , ..., -0.32241133,
        -0.45623422, -0.1330979 ],
       [-0.11743882,  0.7670165 ,  0.04623953, ..., -0.32241133,
        -0.45623422, -0.1330979 ],
       [-0.02283244, -0.18045346,  1.7104937 , ..., -0.32241133,
        -0.45623422, -0.1330979 ]], dtype=float32)

In [36]:
# np_labels.shape


In [37]:
trainingdata = []
labels = []
for index,el in full_dataset.iterrows():
    trainingdata.append([sbert_model.encode(el["argument"]), sbert_model.encode(el["key_point"])])
    labels.append(el["label"])
    #print(el["argument"], el["key_point"], el["label"p

np_trainingdata = np.array(trainingdata)
np_labels = np.array(labels)
# print(type(np_trainingdata))
# print(np_trainingdata.shape)
# print(labels)


In [38]:
np_trainingdata.shape

(3458, 2, 768)

In [39]:
np_trainingdata[1]

array([[ 0.05293709,  0.05693178,  0.6169441 , ...,  0.31999168,
        -0.88885194,  0.0495356 ],
       [ 0.18360981,  0.49867436,  0.332739  , ..., -0.05868514,
        -0.71161807, -0.01766074]], dtype=float32)

In [40]:
from tensorflow import keras
from tensorflow.keras.layers import Dense
from keras.utils.vis_utils import plot_model

In [41]:
model = keras.Sequential()

In [42]:
# model.add(Dense(64, activation='softmax', input_shape=(2,768)))
model.add(Dense(768, activation='softmax', input_dim=(1536)))

In [43]:
model.add(Dense(1, activation='linear'))

In [44]:
model.compile(optimizer='sgd', loss='mse')

In [45]:
model.fit(np_test_training, np_labels, epochs=10, verbose=1, batch_size = 1)

Epoch 1/10
3458/3458 [==============================] - 17s 5ms/step - loss: 0.1700
Epoch 2/10
3458/3458 [==============================] - 17s 5ms/step - loss: 0.1693
Epoch 3/10
3458/3458 [==============================] - 17s 5ms/step - loss: 0.1690
Epoch 4/10
3458/3458 [==============================] - 18s 5ms/step - loss: 0.1687
Epoch 5/10
3458/3458 [==============================] - 17s 5ms/step - loss: 0.1678
Epoch 6/10
3458/3458 [==============================] - 17s 5ms/step - loss: 0.1644
Epoch 7/10
3458/3458 [==============================] - 17s 5ms/step - loss: 0.1440
Epoch 8/10
3458/3458 [==============================] - 17s 5ms/step - loss: 0.1275
Epoch 9/10
3458/3458 [==============================] - 18s 5ms/step - loss: 0.1199
Epoch 10/10
3458/3458 [==============================] - 18s 5ms/step - loss: 0.1164


In [46]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 768)               1180416   
                                                                 
 dense_1 (Dense)             (None, 1)                 769       
                                                                 
Total params: 1,181,185
Trainable params: 1,181,185
Non-trainable params: 0
_________________________________________________________________


In [47]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 768)               1180416   
                                                                 
 dense_1 (Dense)             (None, 1)                 769       
                                                                 
Total params: 1,181,185
Trainable params: 1,181,185
Non-trainable params: 0
_________________________________________________________________


In [48]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 768)               1180416   
                                                                 
 dense_1 (Dense)             (None, 1)                 769       
                                                                 
Total params: 1,181,185
Trainable params: 1,181,185
Non-trainable params: 0
_________________________________________________________________


In [49]:
test = np_trainingdata[0:2]
test
test_label = np_labels[0]
test_label


1

In [50]:
full_dataset.iloc[0:2]

,arg_id,key_point_id,label,argument,topic_x,stance_x,key_point,topic_y,stance_y
0,arg_4_121,kp_4_5,1,A real education is about giving students the ...,We should abandon the use of school uniform,1,School uniform is harming the student's self e...,We should abandon the use of school uniform,1
1,arg_4_122,kp_4_5,1,children express themselves through the clothe...,We should abandon the use of school uniform,1,School uniform is harming the student's self e...,We should abandon the use of school uniform,1


In [51]:
print(test.shape)
print(test)

(2, 2, 768)
[[[-0.22546919  0.42217124  1.0890179  ... -0.03572995 -1.1686127
    0.06441975]
  [ 0.18360981  0.49867436  0.332739   ... -0.05868514 -0.71161807
   -0.01766074]]

 [[ 0.05293709  0.05693178  0.6169441  ...  0.31999168 -0.88885194
    0.0495356 ]
  [ 0.18360981  0.49867436  0.332739   ... -0.05868514 -0.71161807
   -0.01766074]]]


In [52]:
# model.predict(test[1])

ValueError: in user code:

    File "C:\Users\hanne\Anaconda3\envs\ExpAI\lib\site-packages\keras\engine\training.py", line 1621, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\hanne\Anaconda3\envs\ExpAI\lib\site-packages\keras\engine\training.py", line 1611, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\hanne\Anaconda3\envs\ExpAI\lib\site-packages\keras\engine\training.py", line 1604, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\hanne\Anaconda3\envs\ExpAI\lib\site-packages\keras\engine\training.py", line 1572, in predict_step
        return self(x, training=False)
    File "C:\Users\hanne\Anaconda3\envs\ExpAI\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\hanne\Anaconda3\envs\ExpAI\lib\site-packages\keras\engine\input_spec.py", line 263, in assert_

In [53]:
test = np_test_training[0:10]
test



array([[-0.22546919,  0.42217124,  1.0890179 , ..., -0.05868514,
        -0.71161807, -0.01766074],
       [ 0.05293709,  0.05693178,  0.6169441 , ..., -0.05868514,
        -0.71161807, -0.01766074],
       [ 0.3017908 , -0.2542112 ,  0.69037217, ..., -0.05868514,
        -0.71161807, -0.01766074],
       ...,
       [ 0.36468557, -0.5400683 ,  0.9855318 , ..., -0.05868514,
        -0.71161807, -0.01766074],
       [ 0.12908995, -0.2196983 ,  1.0085294 , ..., -0.05868514,
        -0.71161807, -0.01766074],
       [ 0.22413963, -0.05974222,  0.3504017 , ..., -0.05868514,
        -0.71161807, -0.01766074]], dtype=float32)

In [58]:
pred = model.predict(np_test_training)

pred

array([[0.2954883 ],
       [0.30174106],
       [0.2659843 ],
       ...,
       [0.03057504],
       [0.06613782],
       [0.0293678 ]], dtype=float32)

In [60]:
pred = pred.flatten()

In [61]:
type(pred)

numpy.ndarray

In [62]:
np_labels.shape

(3458,)

In [63]:
new = np.vstack((pred, np_labels))
new_df = pd.DataFrame(new)
new_df = new_df.transpose()
new_df.columns = ["predicted", "true"]
new_df

,predicted,true
0,0.295488,1.0
1,0.301741,1.0
2,0.265984,1.0
3,0.269281,1.0
4,0.268448,1.0
...,...,...
3453,0.145447,1.0
3454,0.074076,0.0
3455,0.030575,0.0
3456,0.066138,0.0


In [64]:
new_df['predicted_label'] = np.where(new_df['predicted'] >= 0.5, 1, 0)

In [65]:
new_df['match'] = np.where(new_df['true'] == new_df['predicted_label'], 1, 0)

In [66]:
new_df

,predicted,true,predicted_label,match
0,0.295488,1.0,0,0
1,0.301741,1.0,0,0
2,0.265984,1.0,0,0
3,0.269281,1.0,0,0
4,0.268448,1.0,0,0
...,...,...,...,...
3453,0.145447,1.0,0,0
3454,0.074076,0.0,0,1
3455,0.030575,0.0,0,1
3456,0.066138,0.0,0,1


In [67]:
new_df['match'].sum()
accuracy = new_df['match'].sum() / new_df.shape[0]
accuracy

0.8513591671486408

In [68]:
new_df.describe()

,predicted,true,predicted_label,match
count,3458.000000,3458.000000,3458.000000,3458.000000
mean,0.238096,0.213418,0.160787,0.851359
std,0.252177,0.409780,0.367387,0.355786
min,0.008892,0.000000,0.000000,0.000000
25%,0.049180,0.000000,0.000000,1.000000
50%,0.133358,0.000000,0.000000,1.000000
75%,0.310293,0.000000,0.000000,1.000000
max,0.858490,1.000000,1.000000,1.000000


In [69]:
df_one = new_df[new_df["true"] == 1]

In [70]:
df_one

,predicted,true,predicted_label,match
0,0.295488,1.0,0,0
1,0.301741,1.0,0,0
2,0.265984,1.0,0,0
3,0.269281,1.0,0,0
4,0.268448,1.0,0,0
...,...,...,...,...
3411,0.090499,1.0,0,0
3419,0.062535,1.0,0,0
3424,0.018041,1.0,0,0
3432,0.106757,1.0,0,0


In [71]:
df_zeros = new_df[new_df["true"] == 0]
df_one.describe()

,predicted,true,predicted_label,match
count,738.000000,738.0,738.000000,738.000000
mean,0.520429,1.0,0.528455,0.528455
std,0.287927,0.0,0.499528,0.499528
min,0.009913,1.0,0.000000,0.000000
25%,0.267011,1.0,0.000000,0.000000
50%,0.568339,1.0,1.000000,1.000000
75%,0.813660,1.0,1.000000,1.000000
max,0.858490,1.0,1.000000,1.000000


In [ ]:
df_zeros.describe()